In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from datetime import datetime
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/dutch-news-articles/dutch-news-articles.csv')

Let's plot numbers of articles by categories and years

In [ ]:
#extract years from datetime data
df['datetime']=pd.to_datetime (df['datetime'])
df['year']=pd.DatetimeIndex(df['datetime']).year
#manipulate with data to get a dataframe with number of articles by categories and years
year_category_df=df.groupby(['year','category']).nunique()
year_category_df.reset_index(inplace=True)
year_category_df=year_category_df.pivot(index='year',columns='category',values='url')
#simply plot the pivot dataframe
year_category_df.plot(figsize=(30,10))

Let's generate a wordcloud to undestand most popular words in articles

In [ ]:
#take 1/10 of the content from random rows (time consuming...)
random=np.random.randint(0,len(df.content), size=int(len(df.content)/10))
#take these rows from the content and sum it up
text = df.content[random].sum()

#import packages and make a wordcloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(text)
plt.figure(figsize=(40, 30))
plt.imshow(wordcloud) 
plt.axis("off")

Let's check ratio of words in the content and plot 20 most popular words 

In [ ]:
from nltk import FreqDist
import nltk
#tokenize text into separate words
words = nltk.tokenize.word_tokenize(text)
#make frequency dictionary of the words
fdist = FreqDist(words)
#transform data into df, drop dots and commas, calculate ratio of frequency of each word and plto the data
fdf=pd.DataFrame(fdist, index=['freq']).transpose()
fdf.sort_values(by='freq',ascending=False,inplace=True)
fdf=fdf.drop(axis=0,labels=['.',','])
fdf['ratio']=fdf['freq']/len(words)
fdf['word']=fdf.index
fdf[0:20].plot(x='word',y='ratio',kind='bar',figsize=(10,10),title='20 most popular words ')
plt.xticks(rotation=45)

And a list of 10 least popular words

In [ ]:
fdf.sort_values(by='freq')[['word','freq']].head(n=10)

I am wondering is there correlation between word length and its frequency?

In [ ]:
fdf['length']=[len(fdf['word'][i]) for i in range(len(fdf['word']))]
#drop outliers
fdf1=fdf[fdf['length']<=100]
#plot the scatterplot
plt.figure(figsize=(20,10))
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.scatter(y=fdf1['freq'],x=fdf1['length'],c=fdf1['ratio'])

Let's look on the longest words

In [ ]:
fdf.reset_index(inplace=True)
fdf.sort_values(by='length')['word'].tail(n=20)